In [1]:
import os
import pandas as pd
import numpy as np
import sys
# sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
sys.path.append('C:/Users/murbanek/CUSP/DOT/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
# os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')
os.chdir('C:/Users/murbanek/CUSP/DOT')
%matplotlib inline

## 1. Get the sample of parsed AVL data.
Clean MTA prefixes from trip_id

In [2]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('1203_cleaned.csv')
bustime.drop('Unnamed: 0',axis=1,inplace=True)

# beware!  Bonan is still working on organizing the extract files.  these columns may change.

bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime['shape_id'] = bustime['shape_id'].str.replace('MTA_','')

bustime.set_index(['route','trip_id','trip_date','vehicle_id'],inplace=True,drop=True)
bustime.set_index('timestamp',append=True,drop=False,inplace=True)

# for demonstration, use a subset. Just get data one day.
tripDateLookup = "2015-12-03"
bustime = bustime.xs((tripDateLookup),level=(2),drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
bustime['timestamp'] = bustime['timestamp'].apply(ttools.parseActualTime,tdate='2015-12-03')

bustime['stop_dist_on_trip'] = bustime['stop_dist_on_trip'].convert_objects(convert_numeric=True)
bustime['dist_from_stop'] = bustime['dist_from_stop'].convert_objects(convert_numeric=True)
# add calculated column for distance along shape
bustime['veh_dist_along_trip'] = bustime['stop_dist_on_trip'] - bustime['dist_from_stop']

print 'Finished loading BusTime data, parsing timestamps, and slicing ONE DAY.'

Finished loading BusTime data, parsing timestamps, and slicing ONE DAY.


C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


## 2. calculate speed and duration between each reported location, based on distance fields.

In [3]:
bg = bustime.groupby(level=(0,1,2,3))
bustime['seg_duration'] = bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1)
bustime['seg_speed'] = bg['veh_dist_along_trip'].diff()/(bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1))

In [70]:
bustime['seg_dist'] = bg['veh_dist_along_trip'].diff()

In [86]:
results_df = pd.DataFrame(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max','stopped_count'])
for d in range(0,11000,200):
    subset = bustime.query('shape_id == "B1O0243" & veh_dist_along_trip >= @d & seg_dist <= 1000').groupby(level=(0,1,2,3)).first()['seg_speed']
    descr_stats = subset.describe()
    descr_stats.name = d
    descr_stats['stopped_count'] = sum(subset<0.5)
    results_df = results_df.append(descr_stats)

In [87]:
results_df

,count,mean,std,min,25%,50%,75%,max,stopped_count
0,137,2.275368,2.985534,-0.674704,0.000000,0.423437,3.910775,14.025313,69
200,136,4.912837,2.805865,0.000000,2.782087,4.653790,6.685966,14.025313,3
400,136,5.372742,2.763364,0.000000,3.219859,5.232031,7.404561,14.025313,2
600,136,5.622236,2.852331,0.000000,3.349687,5.535916,7.722568,14.025313,2
800,136,5.518482,2.742214,0.000000,3.349687,5.402992,7.467865,12.213125,2
1000,136,4.991556,2.466484,0.000000,3.265139,5.045152,6.581860,10.309683,2
1200,136,4.265790,1.983266,0.000000,2.935350,4.116277,5.274459,9.357755,2
1400,136,3.952325,1.786834,0.000000,2.879703,3.873579,4.672652,8.536349,2
1600,136,3.433250,1.735451,0.000000,2.318568,3.166537,4.339306,8.536349,2
1800,134,2.205000,1.228510,0.000000,1.267244,1.961154,2.973830,6.718387,4
